# lgbm-HyperOptu 데모

## 라이브러리 import 및 설정


In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import lightgbm as lgb
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin
from sklearn.model_selection import  train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

rcParams['figure.figsize'] = (16, 8)       #그림 사이즈 지정
plt.style.use('fivethirtyeight')           #그림 기본 스타일
pd.set_option('max_columns', 100)          #칼럼 100개 허용
pd.set_option("display.precision", 4)      #소수점 4자리허용
warnings.simplefilter('ignore')            #경고분 무시

## 학습데이터 로드


In [ ]:
feature_file = 'C:\\Users\\USER\\Desktop\\Dataset\\DataInput\\feature.csv'
sample_file  = 'C:\\Users\\USER\\Desktop\\Dataset\\DataInput\\sample_submission.csv'
tst_file     = 'C:\\Users\\USER\\Desktop\\Dataset\\DataInput\\test.csv'
trn_file     = 'C:\\Users\\USER\\Desktop\\Dataset\\DataInput\\train.csv'

df= pd.read_csv(feature_file,index_col=0)        #학습/시험 데이터 
print(df.shape)                                  #(6113, 32)
df.head()                                        #위 5개            

In [4]:
y = df.iloc[:,31].values[:4280]                  #학습_종속
df.drop(df.columns[[31]], axis=1, inplace=True)  #독립을 위한 종속 제거
trn = df.iloc[:4280].values                      #학습데이터_독립 4280개
tst = df.iloc[4280:].values                      #시험데이터_독립 1883개
print(y.shape, trn.shape, tst.shape)             #(4280,) (4280, 31) (1833, 31)

(4280,) (4280, 31) (1833, 31)


## 학습/검증 데이터 구분
학습 후 모델의 예측/분류 정확도 계산을 위한 검증데이터 분리

#### [Hold-out Validation]

In [ ]:
X_trn, X_val, y_trn, y_val = train_test_split(trn, y, test_size=.2, random_state=seed)

## Hyperparameter Tuning

In [5]:
params = {
    "objective": "multiclass",
    "n_estimators": 400,
    "subsample_freq": 1,
    "random_state": seed,
    "n_jobs": -1
}
#고정 파라미터

space = {
    "learning_rate": hp.loguniform("learning_rate", np.log(0.01), np.log(0.2)),
    "num_iterations":  hp.choice("num_iterations", [100, 200, 300, 400, 500, 600,800,1000]),
    "num_leaves": hp.choice("num_leaves", [16, 32, 64, 127, 254]),
    "min_child_samples": hp.choice("min_child_samples", [5, 10, 15, 20, 25, 30, 35,40,45,55]),
    "colsample_bytree": hp.quniform("colsample_bytree", .3, .9, 0.1),
    "subsample": hp.quniform("subsample", .3, .9, 0.1)
}
#변경 파라미터_넓은 범위와 여러 파라미터 사용시 시간 급증 (주의)

# loguniform: 로그함수를 이용해 확률 분포 내에서의 값으로 test
# choice: 리스트 값 중 하나로 선택하여 test
# quniform: start와 stop사이에서 일정 간격으로 값을 주어 test

In [6]:
def objective(hyperparams):
    model = lgb.LGBMClassifier(**params, **hyperparams)
    model.fit(X=X_trn, y=y_trn,
              eval_set=[(X_val, y_val)],
              eval_metric="multi_logloss",#멀티클래스_로그 손실함수
              early_stopping_rounds=10,
              verbose=False)
    score = model.best_score_["valid_0"]["multi_logloss"]
    #측정한 파라미터 중단됐을 때 최고의 성능점수

    return {'loss': score, 'status': STATUS_OK, 'model': model}

trials = Trials()
best = fmin(fn=objective, space=space, trials=trials, algo=tpe.suggest, max_evals=10, verbose=1)
# fn=최소화하고 싶은 함수와 파라미터(space), 
# trials=실행 객체변수, tpe.sugges=알고리즘 선택
# max_evals=서치 횟수 

  0%|                                                                           | 0/10 [00:00<?, ?trial/s, best loss=?]

C:\Users\USER\anaconda3\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



100%|███████████████████████████████████████████████| 10/10 [00:20<00:00,  2.01s/trial, best loss: 0.21915510262775925]


In [58]:
hyperparams = space_eval(space, best)   #검색하여 나온 최적의 파라미터
n_best = trials.best_trial['result']['model'].best_iteration_
params.update(hyperparams)
print(params)                           #업데이트된 파라미터

{'objective': 'multiclass', 'boosting_type': 'gbdt', 'subsample_freq': 1, 'random_state': 150, 'n_jobs': -1, 'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.0662586865182318, 'min_child_samples': 45, 'n_estimators': 200, 'num_iterations': 300, 'num_leaves': 31, 'subsample': 0.6000000000000001}


#1
{'objective': 'multiclass', 'subsample_freq': 1, 'random_state': 150, 'n_jobs': -1,   
'boosting_type': 'dart', 'colsample_bytree': 0.6000000000000001,   
'learning_rate': 0.028778270063003097, 'n_estimators': 100, 'num_iterations': 1000,  
'num_leaves': 31, 'subsample': 0.6000000000000001}
90.--

#2
{'objective': 'multiclass', 'subsample_freq': 1, 'random_state': 150,  
'n_jobs': -1, 'boosting_type': 'gbdt', 'colsample_bytree': 0.7000000000000001,   
'learning_rate': 0.06316437855450058, 'n_estimators': 600, 'num_leaves': 10,  
'subsample': 0.6000000000000001}  
91.308

#3 
{'objective': 'multiclass', 'boosting_type': 'gbdt', 'subsample_freq': 1,   
 'random_state': 150, 'n_jobs': -1, 'colsample_bytree': 0.7000000000000001,   
  'learning_rate': 0.0662586865182318, 'min_child_samples': 45, 'n_estimators': 200,    
 'num_iterations': 300, 'num_leaves': 31, 'subsample': 0.6000000000000001}

## Stratified K-Fold Cross Validation
*Stratified N-Fold CV: N-Fold CV에서 각각의 폴드에서 종속변수의 분포가 동일하도록 폴드를 나누는 방식.
현재 사용하는 데이터처럼 분류학습에서 종속변수의 범주의 분포가 균일하지 않을 때 사용된다.

In [ ]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

### [범주형 > 수치형 변환_종속변수]

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

label_str=y
label_int=le.fit_transform(label_str).astype('int')

## LightGBM 모델 학습

In [ ]:
#93.3216% seed1022
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
#(cv.split(),1)
# -  StratifiedKFold 경우, trn,y 독립과 종속(동일 분포를 위해) 인자
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(**params)
    clf.fit(trn[i_trn], y[i_trn],
            eval_set=[(trn[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=10)
    
    p_val[i_val, :] = clf.predict_proba(trn[i_val])
    #확률로 예측값 초가화
    p_tst += clf.predict_proba(tst) / n_fold
    #평균 효과를 위해 먼저 폴드 개수만큼 나누어 초기화

In [ ]:
** ( 범주값 출력: clf.pedict(), 확률값출력: clf.predict_proba() ) **

In [ ]:
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

## 피처 중요도 시각화
여러 모델 시각화 대신 각 모델에서 손실 감소에 기여한 피처들의 중요도를 보기 위한 과정

In [ ]:
imp = pd.DataFrame({'feature': df.columns, 'importance': clf.feature_importances_})
imp = imp.sort_values('importance').set_index('feature')
imp.plot(kind='barh')

### [수치형 > 범주형 변환_종속변수]

In [ ]:
pt=np.zeros(shape=(1833,))
pt = pt.astype(np.int64)
for x in range(0,1833):
    pt[x] = np.argmax(p_tst[x,:])
target=le.inverse_transform(pt)
target

## 제출 파일 생성

In [ ]:
#sub = pd.read_csv(sample_file)
#sub['class']
sub = pd.read_csv(sample_file)
sub['class'] = p_tst
sub.to_csv('C:\\Users\\USER\\Desktop\\Dataset\\sub\\lgbmHyper-sub.csv',index=False,header=False)